## Imports

In [1]:
import requests
from bs4 import BeautifulSoup

In [87]:
embrapa_url = "http://vitibrasil.cnpuv.embrapa.br/index.php"
re = requests.get(embrapa_url)
soup = BeautifulSoup(re._content)
# páginas
pages = {}
for i in soup.find_all('button'):
    texto = i.text
    value = '' if 'value' not in i.attrs.keys() else i.attrs['value']
    pages[texto] = {
        'link': f'{embrapa_url}?opcao={value}',
        'has_subpages': 'No', 
        'sub_pages': {},
        'download_link': {}
        }
    print(texto, value)

Apresentação opt_01
Produção opt_02
Processamento opt_03
Comercialização opt_04
Importação opt_05
Exportação opt_06
Publicação opt_07


In [103]:
pages

{'Apresentação': {'link': 'http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_01',
  'has_subpages': 'No',
  'sub_pages': {},
  'download_link': ['http://vitibrasil.cnpuv.embrapa.br/download/apresenta.pdf']},
 'Produção': {'link': 'http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_02',
  'has_subpages': 'No',
  'sub_pages': {},
  'download_link': ['http://vitibrasil.cnpuv.embrapa.br/download/Producao.csv']},
 'Processamento': {'link': 'http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_03',
  'has_subpages': 'yes',
  'sub_pages': {'Viníferas': 'http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_03&subopcao=subopt_01',
   'Americanas e híbridas': 'http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_03&subopcao=subopt_02',
   'Uvas de mesa': 'http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_03&subopcao=subopt_03',
   'Sem classificação': 'http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_03&subopcao=subopt_04'},
  'download_link': ['http://vitibrasil.cnpuv.

In [96]:
# Sub Páginas

for key, values in pages.items():
    print(key)
    re = requests.get(values['link'])
    soup = BeautifulSoup(re._content)
    subpages_dict = {}
    for i in soup.find_all('button'):
        text = i.text
        btn_value = '' if 'value' not in i.attrs.keys() else i.attrs['value']
        if 'subopt' in btn_value:
            print(f"subopt: {btn_value}")
            values['has_subpages'] = 'yes' 
            subpages_dict[text] = f"{values['link']}&subopcao={btn_value}"
    values['sub_pages'] = subpages_dict


Apresentação
Produção
Processamento
subopt: subopt_01
subopt: subopt_02
subopt: subopt_03
subopt: subopt_04
Comercialização
Importação
subopt: subopt_01
subopt: subopt_02
subopt: subopt_03
subopt: subopt_04
subopt: subopt_05
Exportação
subopt: subopt_01
subopt: subopt_02
subopt: subopt_03
subopt: subopt_04
Publicação


In [108]:
# Download Links
for key, values in pages.items():
    if key in ['Apresentação', 'Publicação']: continue
    if values['has_subpages'] == 'No':
        re = requests.get(values['link'])
        soup = BeautifulSoup(re._content)
        for link_element in soup.find_all('a', class_='footer_content'):
            if 'href' in link_element.attrs.keys() and 'DOWNLOAD' in link_element.text:
                values['download_link'] = [embrapa_url.replace('index.php', link_element['href'])]
                # Fazendo a requisição para baixar o conteúdo do CSV
                if 'pdf' in link_element['href']: continue
                response = requests.get(embrapa_url.replace('index.php', link_element['href']))
                # Checando se o download foi bem-sucedido
                if response.status_code == 200:
                    # Salvando o conteúdo do arquivo
                    name = link_element['href'].split('/')[-1]
                    with open(f"data/data_{name}", "wb") as file:
                        file.write(response.content)
                    print("Arquivo CSV baixado e salvo com sucesso.")
                else:
                    print(f"Erro ao baixar o arquivo: {response.status_code}")
    else:
        download_link = []
        for subpages in values['sub_pages'].values():
            re = requests.get(subpages)
            soup = BeautifulSoup(re._content)
            for link_element in soup.find_all('a', class_='footer_content'):
                if 'href' in link_element.attrs.keys() and 'DOWNLOAD' in link_element.text:
                    download_link.append(embrapa_url.replace('index.php', link_element['href']))
                    # Fazendo a requisição para baixar o conteúdo do CSV
                    if 'pdf' in link_element['href']: continue
                    response = requests.get(embrapa_url.replace('index.php', link_element['href']))
                    # Checando se o download foi bem-sucedido
                    if response.status_code == 200:
                        # Salvando o conteúdo do arquivo
                        name = link_element['href'].split('/')[-1]
                        with open(f"data/data_{name}", "wb") as file:
                            file.write(response.content)
                        print("Arquivo CSV baixado e salvo com sucesso.")
                    else:
                        print(f"Erro ao baixar o arquivo: {response.status_code}")
        values['download_link'] = download_link



Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.
Arquivo CSV baixado e salvo com sucesso.


In [75]:
soup.find_all('a', class_='footer_content')

[<a class="footer_content" href="download/Producao.csv" target="_blank">
 <img alt="" height="16" src="figures/download.png" width="16"/>
 <span class="spn_small">DOWNLOAD</span>
 </a>,
 <a class="footer_content" href="#_top">
 <img alt="" height="16" src="figures/topo.png" width="16"/>
 <span class="spn_small">TOPO</span>
 </a>]

In [50]:
# Fazendo a requisição para baixar o conteúdo do CSV
response = requests.get('http://vitibrasil.cnpuv.embrapa.br/download/Producao.csv')

# Checando se o download foi bem-sucedido
if response.status_code == 200:
    # Salvando o conteúdo do arquivo
    with open("arquivo_baixado.csv", "wb") as file:
        file.write(response.content)
    print("Arquivo CSV baixado e salvo com sucesso.")
else:
    print(f"Erro ao baixar o arquivo: {response.status_code}")


Arquivo CSV baixado e salvo com sucesso.


In [1]:
# app.py

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///example.db'
db = SQLAlchemy(app)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)

In [ ]:
# app.py, blueprint

from flask import Blueprint, render_template

auth_bp = Blueprint('auth', __name__)

@auth_bp.route('/login')
def login():
    return render_template('login.html')

# Integrando o blueprint no aplicativo principal
app.register_blueprint(auth_bp, url_prefix='/auth')

In [ ]:
# main.py

from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Item(BaseModel):
    name: str
    description: str = None
    price: float
    tax: float = None

In [ ]:
# main.py

from fastapi import Depends, FastAPI, HTTPException
from fastapi.security import OAuth2PasswordBearer

app = FastAPI()

oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

# Uso de dependência avançada
def get_current_user(token: str = Depends(oauth2_scheme)):
    credentials_exception = HTTPException(
        status_code=401,
        detail="Could not validate credentials",
        headers={"WWW-Authenticate": "Bearer"}
    )
    # Lógica para validar o token e obter o usuário atual
    return current_user

@app.get("/users/na")
async def read_user_na(current_user: User = Depends(get_current_user)):
    return {"username": current_user.username}